In [33]:
import torch
import torch.nn as nn
import pandas as pd
import warnings
import os
warnings.filterwarnings('ignore')
parent_path = 'C:\\Users\\ismai\\Documents\\Comment-Section-UI\\angular-interactive-comments-section\\flask\\data'

In [34]:

file_name = 'fox-new-york-could-face'
file = os.path.join(parent_path, f'{file_name}.csv')
df = pd.read_csv(file)

In [35]:
df.head(5)

,Unnamed: 0,comment,time,username,id,timestamp,time_diff
0,321,And so ... the 'lawful' cheating begins!,"22 Dec, 2023 03:28 AM",senorbiggles,322,2023-12-22 03:28:00,0 days 00:00:00
1,875,And so ... the 'lawful' cheating begins!,"22 Dec, 2023 03:28 AM",senorbiggles,876,2023-12-22 03:28:00,0 days 00:00:00
2,296,Democrats desperately rigging the elections in...,"22 Dec, 2023 03:28 AM",HarryBaals404,297,2023-12-22 03:28:00,0 days 00:00:00
3,850,Democrats desperately rigging the elections in...,"22 Dec, 2023 03:28 AM",HarryBaals404,851,2023-12-22 03:28:00,0 days 00:00:00
4,144,Democrats can’t win on their achievements so t...,"22 Dec, 2023 03:29 AM",HarryBaals404,145,2023-12-22 03:29:00,0 days 00:01:00


In [36]:
list(df.iloc[1][1:])

["And so ... the 'lawful' cheating begins!",
 '22 Dec, 2023 03:28 AM',
 'senorbiggles',
 876,
 '2023-12-22 03:28:00',
 '0 days 00:00:00']

In [37]:
def get_so_far_comments(ith_comment):
    # row_index = df.loc[df['id'] == comment_id].index[0]
    ids = df.iloc[0:ith_comment+1]['id']
    return list(ids)


In [38]:
# get_so_far_comments(1)

In [39]:
df_trend = pd.read_csv(os.path.join(parent_path, f'{file_name}-trends.csv'))

In [40]:
df_trend.shape

(1064, 10)

In [41]:
def get_single_trend_score(trend):
    print("label: ", trend)
    return list(df_trend[trend])

In [42]:
# get_single_trend_score('toxic')

In [43]:
previous_emotion = []

In [44]:
# import random

# def get_emotion(escores, id):
#     # print("id")
#     if id == 0:
#         previous_emotion = escores

#     if id%100 == 0:
#         X = escores[27]*0.6
#         escores[27] = escores[27] - X
#         y = X*(random.sample(range(0, 100), 1)[0]/100)
#         random_numbers = random.sample(range(0, 27), 3)
#         escores[random_numbers[0]] = X*0.2
#         escores[random_numbers[1]] = X*0.7
#         escores[random_numbers[2]] = X*0.1
#         previous_emotion = escores
#         return escores
    
#     if id%200 == 0:
#         X = escores[27]*0.6
#         escores[27] = escores[27] - X
#         y = X*(random.sample(range(0, 100), 1)[0]/100)
#         random_numbers = random.sample(range(0, 27), 3)
#         escores[random_numbers[0]] = X*0.1
#         escores[random_numbers[1]] = X*0.6
#         escores[random_numbers[2]] = X*0.3
#         previous_emotion = escores
#         return escores
    
#     return previous_emotion


In [45]:
from common_function import get_emotions, get_sentiment_dict
from score_aggregation import get_polarity_score, find_indices

emotions = list(get_emotions().values())
sentiment_dict = get_sentiment_dict()

In [ ]:
pos_indices = find_indices(sentiment_dict['positive'], emotions)
neg_indices = find_indices(sentiment_dict['negative'], emotions)
amb_indices = find_indices(sentiment_dict['ambiguous'], emotions)
neu_indices = find_indices(sentiment_dict['neutral'], emotions)

def get_emotion(escores, id):
    # print(escores)

    if id < 100:
        X = escores[27]*0.4
        escores[27] = escores[27] - X
        for i in amb_indices:
            escores[i] += (X*0.10)

    elif id > 100 and id < 200:
        X = escores[27]*0.5
        escores[27] = escores[27] - X
        for i in pos_indices:
            escores[i] += (X*0.15)

    elif id > 200 and id < 400:
        X = escores[27]*0.6
        escores[27] = escores[27] - X
        for i in pos_indices:
            escores[i] += (X*0.10)

    elif id > 400 and id < 600:
        X = escores[27]*0.6
        escores[27] = escores[27] - X
        for i in pos_indices:
            escores[i] += (X*0.15)

    elif id > 600 and id < 800:
        X = escores[27]*0.65
        escores[27] = escores[27] - X
        for i in neg_indices:
            escores[i] += (X*0.10)

    elif id > 800:
        X = escores[27]*0.7
        escores[27] = escores[27] - X
        for i in neg_indices:
            escores[i] += (X*0.15)

    # print(escores)
    escores =  [float(i)/sum(escores) for i in escores]

    # print("SUM:", sum(escores))
    return escores

In [47]:
def get_trends(tscores, id):

    # print(escores)
    if id < 100:
        X = tscores[7]*0.5
        tscores[7] = tscores[7] - X
        for i in [0, 7]:
            tscores[i] += (X*0.10)

    elif id > 100 and id < 200:
        X = tscores[7]*0.57
        tscores[7] = tscores[7] - X
        for i in [0, 7]:
            tscores[i] += (X*0.10)

    elif id > 200 and id < 500:
        X = tscores[7]*0.6
        tscores[7] = tscores[7] - X
        for i in [0, 7]:
            tscores[i] += (X*0.10)

    elif id > 500 and id < 700:
        X = tscores[7]*0.5
        tscores[7] = tscores[7] - X
        for i in [1, 2, 3, 5]:
            tscores[i] += (X*0.10)

    elif id > 700 and id < 800:
        X = tscores[7]*0.6
        tscores[7] = tscores[7] - X
        for i in [1, 2, 3, 5]:
            tscores[i] += (X*0.10)

    elif id > 800 and id < 900:
        X = tscores[7]*0.65
        tscores[7] = tscores[7] - X
        for i in [4, 6]:
            tscores[i] += (X*0.10)

    elif id > 900:
            X = tscores[7]*0.7
            tscores[7] = tscores[7] - X
            for i in [4, 6]:
                tscores[i] += (X*0.10)
    # print(escores)
    tscores =  [float(i)/sum(tscores) for i in tscores]

    # print("SUM:", sum(escores))
    return tscores

In [48]:
from score_aggregation import get_emotion_trends_polarity

def getData(index):
    ids = get_so_far_comments(ith_comment=index)
    print("Index: ", index)
    print("So far Comments: ", ids)
    escores, tscores, polarity = get_emotion_trends_polarity(ids)
    return escores, tscores, polarity

In [49]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import json

app = Flask(__name__)
cors = CORS(app, resources={r"/api/*": {"origins": "*"}})

@app.route('/api/new-comment', methods=['POST'])
def welcome():
    comment = request.json['content']
    img = request.json['img']
    isYou = request.json['isYou']
    score = request.json['score']
    username = request.json['username']
    print("new: ", comment)
    return 'Successfully comment added!'

@app.route('/api/data/<int:id>', methods=['GET'])
def getApiData(id: int):
    escores, tscores, polarity = getData(id)
    escores = get_emotion(escores, id)
    tscores = get_trends(tscores, id)
    polarity = get_polarity_score(escores)
    return json.dumps({"escores": escores, "tscores": tscores, "polarity": polarity})

@app.route('/api/data/single-trend/<string:label>', methods=['GET'])
def getSingleTrend(label: str):
    return json.dumps({"singletrend": get_single_trend_score(label)})

if __name__ == '__main__':
    app.run(host='127.0.0.1', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


Index:  0
So far Comments:  [322]


127.0.0.1 - - [08/Jan/2024 00:34:15] "GET /api/data/0 HTTP/1.1" 200 -


Index:  0
So far Comments:  [322]


...

127.0.0.1 - - [08/Jan/2024 00:36:22] "GET /api/data/565 HTTP/1.1" 200 -


Index:  622
So far Comments:  [322, 876, 297, 851, 145, 699, 110, 664, 61, 615, 420, 974, 323, 877, 298, 852, 840, 286, 111, 665, 700, 146, 62, 841, 287, 616, 607, 53, 668, 114, 147, 701, 51, 471, 1025, 605, 484, 1038, 992, 438, 140, 694, 373, 927, 958, 404, 207, 761, 1039, 485, 554, 410, 762, 208, 964, 704, 150, 583, 29, 617, 63, 874, 320, 470, 1024, 46, 600, 986, 432, 1004, 450, 547, 121, 675, 122, 676, 103, 657, 149, 445, 999, 703, 251, 805, 889, 335, 214, 768, 34, 588, 980, 426, 511, 1029, 54, 475, 608, 512, 513, 358, 912, 336, 890, 252, 806, 104, 123, 658, 677, 953, 399, 264, 818, 129, 683, 447, 1001, 1030, 476, 1003, 449, 498, 989, 1052, 435, 380, 934, 555, 1, 929, 375, 431, 985, 300, 854, 807, 253, 337, 891, 509, 1063, 499, 1053, 698, 144, 44, 598, 338, 892, 627, 669, 115, 1046, 492, 73, 190, 744, 684, 954, 400, 130, 1012, 458, 745, 191, 218, 772, 172, 726, 746, 192, 193, 747, 687, 133, 401, 158, 712, 955, 770, 216, 717, 163, 829, 275, 386, 848, 294, 940, 84, 638, 403, 957, 610,

127.0.0.1 - - [08/Jan/2024 00:36:27] "GET /api/data/622 HTTP/1.1" 200 -
127.0.0.1 - - [08/Jan/2024 00:36:29] "GET /api/data/683 HTTP/1.1" 200 -
127.0.0.1 - - [08/Jan/2024 00:36:29] "GET /api/data/718 HTTP/1.1" 200 -


Index:  754
So far Comments:  [322, 876, 297, 851, 145, 699, 110, 664, 61, 615, 420, 974, 323, 877, 298, 852, 840, 286, 111, 665, 700, 146, 62, 841, 287, 616, 607, 53, 668, 114, 147, 701, 51, 471, 1025, 605, 484, 1038, 992, 438, 140, 694, 373, 927, 958, 404, 207, 761, 1039, 485, 554, 410, 762, 208, 964, 704, 150, 583, 29, 617, 63, 874, 320, 470, 1024, 46, 600, 986, 432, 1004, 450, 547, 121, 675, 122, 676, 103, 657, 149, 445, 999, 703, 251, 805, 889, 335, 214, 768, 34, 588, 980, 426, 511, 1029, 54, 475, 608, 512, 513, 358, 912, 336, 890, 252, 806, 104, 123, 658, 677, 953, 399, 264, 818, 129, 683, 447, 1001, 1030, 476, 1003, 449, 498, 989, 1052, 435, 380, 934, 555, 1, 929, 375, 431, 985, 300, 854, 807, 253, 337, 891, 509, 1063, 499, 1053, 698, 144, 44, 598, 338, 892, 627, 669, 115, 1046, 492, 73, 190, 744, 684, 954, 400, 130, 1012, 458, 745, 191, 218, 772, 172, 726, 746, 192, 193, 747, 687, 133, 401, 158, 712, 955, 770, 216, 717, 163, 829, 275, 386, 848, 294, 940, 84, 638, 403, 957, 610,

127.0.0.1 - - [08/Jan/2024 00:36:45] "GET /api/data/754 HTTP/1.1" 200 -
127.0.0.1 - - [08/Jan/2024 00:36:47] "GET /api/data/790 HTTP/1.1" 200 -
127.0.0.1 - - [08/Jan/2024 00:36:48] "GET /api/data/862 HTTP/1.1" 200 -
127.0.0.1 - - [08/Jan/2024 00:36:48] "GET /api/data/827 HTTP/1.1" 200 -
127.0.0.1 - - [08/Jan/2024 00:36:48] "GET /api/data/891 HTTP/1.1" 200 -
